## Practical Example of Content Moderation:

In [1]:
import os
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver


/opt/anaconda3/envs/langchain_ai/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
#Set up your OpenAi API key
from httpx import stream
from langgraph.graph import state


openai_api_key = os.getenv("OPENAI_API_KEY")

#Initialize the model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, openai_api_key=openai_api_key, max_tokens=50)

#Define the state structure
class State(TypedDict):
    input: str
    draft_content: str

#Define  node functions
def create_draft(state: State):
    print("--- Generating Draft with ChatOpenAI ---")
    #Prepare the prompt for generating the blog content
    prompt = f"""
    Generate a blog post about the topic: {state["input"]}
    """
    #Call the llm instance to generate the draft content
    response = llm.invoke([{"role": "user", "content": prompt}])
    #Extract the generated content
    state["draft_content"] = response.content
    print(f"Draft generated: {state['draft_content']}")
    return state

def review_draft(state: State):
    print("--- Reviewing Draft ---")
    print(f"Draft for review:\n {state['draft_content']}")
    return state


def publish_content(state: State):
    print("--- Publishing Content ---")
    print(f"Published Content:\n {state['draft_content']}")
    return state


#Build the graph 
builder = StateGraph(State)
builder.add_node("create_draft", create_draft)
builder.add_node("review_draft", review_draft)
builder.add_node("publish_content", publish_content)

#Define flow
builder.add_edge(START, "create_draft")
builder.add_edge("create_draft", "review_draft")
builder.add_edge("review_draft", "publish_content")
builder.add_edge("publish_content", END)


#Set up memory and breakpoints
memory = MemorySaver()
graph = builder.compile(checkpointer=memory, interrupt_before=["publish_content"])

#Run the graph
config = {"configurable": {"thread_id": "thread-1"}}
initial_input = {"input": "The importance of AI in modern content creation"}

#Run the first part until the review step
thread = {"configurable": {"thread_id": "1"}}
for event in graph.stream(initial_input,thread,stream_mode="values"):
    print(event)

#Pausing for human review
user_approval = input("Do you approve the draft for publishing? (yes/no/modification)")
if user_approval.lower() == "yes":
    #Process to publish content
    for event in graph.stream(None,thread,stream_mode="values"):
        print(event)

elif user_approval.lower()  == "modification":
    #Aloww modification of the draft
    updated_draft = input("Please modify the draft content:\n")
    memory.update({"draft_content": updated_draft}) 
    print("Draft updatet by the editor")
    #Continue to publishing with the modified draft
    for event in graph.stream(None,thread,stream_mode="values"):
        print(event)

else:
    print("Execution halted by user")
                      

    


{'input': 'The importance of AI in modern content creation'}
--- Generating Draft with ChatOpenAI ---
Draft generated: # The Importance of AI in Modern Content Creation

In an age where information is abundant and attention spans are dwindling, the demand for high-quality, engaging content has never been higher. As brands and creators strive to capture the interest of their audiences,
{'input': 'The importance of AI in modern content creation', 'draft_content': '# The Importance of AI in Modern Content Creation\n\nIn an age where information is abundant and attention spans are dwindling, the demand for high-quality, engaging content has never been higher. As brands and creators strive to capture the interest of their audiences,'}
--- Reviewing Draft ---
Draft for review:
 # The Importance of AI in Modern Content Creation

In an age where information is abundant and attention spans are dwindling, the demand for high-quality, engaging content has never been higher. As brands and creators